In [8]:
import pandas as pd
from pathlib import Path

In [3]:
# Load the diabetic dataset
df = pd.read_csv(r"C:\Users\HOME\Downloads\MediCareNet\data\raw\diabetic_data.csv")

In [4]:
structure = df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [5]:
df.head(5)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [6]:
df.isnull().sum().sort_values(ascending=False)

max_glu_serum               96420
A1Cresult                   84748
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
encounter_id                    0
patient_nbr                     0
number_inpatient                0
number_emergency                0
diag_1                          0
diag_2                          0
number_diagnoses                0
diag_3                          0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [7]:
readmitted_counts = df['readmitted'].value_counts()
readmitted_counts.head()

readmitted
NO     54864
>30    35545
<30    11357
Name: count, dtype: int64

In [10]:
# Paths
input_path = Path("../data/raw/diabetic_data.csv")
output_path = Path("../data/processed/cleaned_data.csv")

In [11]:
# Convert readmitted column to binary target
df['readmitted_30'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

# Drop the original column
df = df.drop(columns=['readmitted'])

In [12]:
# Replace '?' with NaN for analysis
df = df.replace("?", pd.NA)

In [13]:
id_uniqueness = {
    "encounter_id_unique": df['encounter_id'].is_unique,
    "patient_nbr_unique": df['patient_nbr'].is_unique
}

In [14]:
id_uniqueness

{'encounter_id_unique': True, 'patient_nbr_unique': False}

encounter_id is unique, acting like a row ID. patient_nbr is reused but isn’t meaningful without longitudinal modeling.
→ Best practice: Drop both unless you're doing patient-level sequence modeling.

In [ ]:
# 2. Check missing or single-value dominant columns
high_missing_or_dominant = []
for col in ['weight', 'payer_code', 'medical_specialty']:
    top_value = df[col].value_counts(dropna=False).index[0]
    top_pct = df[col].value_counts(normalize=True, dropna=False).iloc[0]
    high_missing_or_dominant.append({
        "Column": col,
        "Top Value": top_value,
        "Top %": f"{top_pct:.2%}"
    })

In [16]:
pd.DataFrame(high_missing_or_dominant)

,Column,Top Value,Top %
0,weight,<NA>,96.86%
1,payer_code,<NA>,39.56%
2,medical_specialty,<NA>,49.08%


These fields have >90% missing or default values — keeping them would add noise.

In [17]:
# 3. Check other columns with >90% same value
over_90_same = []
for col in df.columns:
    top_value = df[col].value_counts(dropna=False).index[0]
    top_pct = df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_pct > 0.90:
        over_90_same.append({
            "Column": col,
            "Top Value": top_value,
            "Top %": f"{top_pct:.2%}"
        })

In [18]:
pd.DataFrame(over_90_same)

,Column,Top Value,Top %
0,weight,<NA>,96.86%
1,max_glu_serum,NaN,94.75%
2,repaglinide,No,98.49%
3,nateglinide,No,99.31%
4,chlorpropamide,No,99.92%
5,glimepiride,No,94.90%
6,acetohexamide,No,100.00%
7,tolbutamide,No,99.98%
8,pioglitazone,No,92.80%
9,rosiglitazone,No,93.75%


Drop or combine these into a single “used_other_drugs” flag during feature engineering if needed.

In [22]:
# Drop ID columns (not useful for prediction)
df = df.drop(columns=['encounter_id', 'patient_nbr'])

# Drop high-missing or single-value columns
to_drop = ['weight', 'payer_code', 'medical_specialty']
df = df.drop(columns=to_drop)

In [24]:
df.isna().sum().sort_values(ascending=False)

max_glu_serum               96420
A1Cresult                   84748
race                         2273
diag_3                       1423
diag_2                        358
diag_1                         21
admission_source_id             0
num_lab_procedures              0
time_in_hospital                0
gender                          0
age                             0
admission_type_id               0
discharge_disposition_id        0
number_inpatient                0
number_emergency                0
number_outpatient               0
num_medications                 0
num_procedures                  0
number_diagnoses                0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide                       0
tolbutamide                     0
pioglitazone                    0
rosiglitazone 

In [25]:
# Get unique values and their counts
unique_values = df['max_glu_serum'].value_counts(dropna=False).reset_index()
unique_values.columns = ['Value', 'Count']
unique_count = df['max_glu_serum'].nunique(dropna=False)

unique_values, unique_count

(  Value  Count
 0   NaN  96420
 1  Norm   2597
 2  >200   1485
 3  >300   1264,
 4)

In [26]:
unique_values = df['A1Cresult'].value_counts(dropna=False).reset_index()
unique_values.columns = ['Value', 'Count']
unique_count = df['A1Cresult'].nunique(dropna=False)

unique_values, unique_count

(  Value  Count
 0   NaN  84748
 1    >8   8216
 2  Norm   4990
 3    >7   3812,
 4)

In [37]:
unique_values = df['gender'].value_counts(dropna=False).reset_index()
unique_values.columns = ['Value', 'Count']
unique_count = df['gender'].nunique(dropna=False)

unique_values, unique_count

(             Value  Count
 0           Female  54708
 1             Male  47055
 2  Unknown/Invalid      3,
 3)

In [38]:
unique_values = df['race'].value_counts(dropna=False).reset_index()
unique_values.columns = ['Value', 'Count']
unique_count = df['race'].nunique(dropna=False)

unique_values, unique_count

(             Value  Count
 0        Caucasian  76099
 1  AfricanAmerican  19210
 2             <NA>   2273
 3         Hispanic   2037
 4            Other   1506
 5            Asian    641,
 6)

In [39]:
df = df[df['gender'] != 'Unknown/Invalid']
df = df[df['race'].notna()]

In [40]:
unique_values = df['gender'].value_counts(dropna=False).reset_index()
unique_values.columns = ['Value', 'Count']
unique_count = df['gender'].nunique(dropna=False)

unique_values, unique_count

(    Value  Count
 0  Female  53575
 1    Male  45917,
 2)

In [ ]:
unique_values = df['race'].value_counts(dropna=False).reset_index()
unique_values.columns = ['Value', 'Count']
unique_count = df['race'].nunique(dropna=False)

unique_values, unique_count

(             Value  Count
 0        Caucasian  76099
 1  AfricanAmerican  19210
 2         Hispanic   2037
 3            Other   1505
 4            Asian    641,
 5)

In [34]:
print(df['diag_1'].isna().sum())
print(df['diag_2'].isna().sum())
print(df['diag_3'].isna().sum())

21
358
1423


In [35]:
df['diag_1'] = df['diag_1'].fillna('Unknown')
df['diag_2'] = df['diag_2'].fillna('Unknown')
df['diag_3'] = df['diag_3'].fillna('Unknown')

In [36]:
# Replace NaN with 'None'
df['max_glu_serum'] = df['max_glu_serum'].fillna('None')
df['A1Cresult'] = df['A1Cresult'].fillna('None')

# Encode values manually
glu_encoding = {'None': 0, 'Norm': 1, '>200': 2, '>300': 3}
a1c_encoding = {'None': 0, 'Norm': 1, '>7': 2, '>8': 3}

df['max_glu_serum'] = df['max_glu_serum'].map(glu_encoding)
df['A1Cresult'] = df['A1Cresult'].map(a1c_encoding)

In [44]:
# Function to map ICD-9 codes to categories
def map_icd_to_category(code):
    try:
        if code == 'Unknown':
            return 'Unknown'
        if code.startswith('E'):
            return 'External Causes'
        if code.startswith('V'):
            return 'Supplemental'
        code = float(code)
        if 390 <= code <= 459 or code == 785:
            return 'Circulatory'
        elif 460 <= code <= 519 or code == 786:
            return 'Respiratory'
        elif 520 <= code <= 579 or code == 787:
            return 'Digestive'
        elif 250 <= code < 251:
            return 'Diabetes'
        elif 800 <= code <= 999:
            return 'Injury'
        elif 710 <= code <= 739:
            return 'Musculoskeletal'
        elif 580 <= code <= 629 or code == 788:
            return 'Genitourinary'
        elif 140 <= code <= 239:
            return 'Neoplasms'
        else:
            return 'Other'
    except:
        return 'Unknown'

# Apply mapping to diag columns
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[f'{col}_category'] = df[col].apply(map_icd_to_category)

# Show a sample of the mapped categories
df[['diag_1', 'diag_1_category', 'diag_2', 'diag_2_category', 'diag_3', 'diag_3_category']].sample(5)


,diag_1,diag_1_category,diag_2,diag_2_category,diag_3,diag_3_category
98930,427,Circulatory,424,Circulatory,414,Circulatory
52808,410,Circulatory,276,Other,276,Other
75659,786,Respiratory,401,Circulatory,250,Diabetes
88061,250.13,Diabetes,790,Other,V58,Supplemental
16541,562,Digestive,553,Digestive,250,Diabetes


In [45]:
# -------------------------
# 8. Save cleaned dataset
# -------------------------

output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)

print(f"Cleaned data saved to: {output_path}")
print(f"Final shape: {df.shape}")

Cleaned data saved to: ..\data\processed\cleaned_data.csv
Final shape: (99492, 48)
